Задание 1. (5 баллов) 
В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf

In [44]:
dostoevsky = open('besy_dostoevsky.txt', encoding='utf-8').read()

In [6]:
from string import punctuation
punctuation += '«»—–…“”'
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from collections import Counter
from nltk.tokenize import sent_tokenize

def normalize(text):
    normalized_text = [word.strip(punctuation) for word 
                       in text.lower().split()]
    normalized_text = [word for word in normalized_text 
                       if word]
    return normalized_text

In [45]:
norm_dostoevsky = normalize(dostoevsky)
vocab_dostoevsky = Counter(norm_dostoevsky)

In [46]:
probas_dostoevsky = Counter({word:c/len(norm_dostoevsky) 
                             for word, c in vocab_dostoevsky.items()})
probas_dostoevsky.most_common(5)

[('и', 0.04316943693335454),
 ('в', 0.02362513917607762),
 ('не', 0.023520757117862254),
 ('что', 0.01769524415460474),
 ('я', 0.017178304437728647)]

In [33]:
def ngrammer (tokens, n=3):
    ngrams = []
    for i in range (0, len(tokens) - n + 1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [146]:
sentences_dostoevsky = [['<start>'] + ['<start>'] + normalize(text) +\
                        ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_dostoevsky = sentences_dostoevsky[:5000]

In [147]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()
trigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
  unigrams_dostoevsky.update(sentence)
  bigrams_dostoevsky.update(ngrammer(sentence, n=2))
  trigrams_dostoevsky.update(ngrammer(sentence))

Чтобы посчитать условную вероятность мы можем поделить количество вхождений триграма на количество вхождений первых двух слов

In [148]:
matrix_dostoevsky = np.zeros((len(bigrams_dostoevsky),
                            len(unigrams_dostoevsky)))

id2bigram_dostoevsky = list(bigrams_dostoevsky)
bigram2id_dostoevsky = {word:i for i, word 
                        in enumerate(id2bigram_dostoevsky)}

id2unigram_dostoevsky = list(unigrams_dostoevsky)
unigram2id_dostoevsky = {word:i for i, word 
                        in enumerate(id2unigram_dostoevsky)}

In [149]:
for trigram in trigrams_dostoevsky:
    word1, word2, word3 = trigram.split()
    words1_2 = word1 + " " + word2
    
    matrix_dostoevsky[
        bigram2id_dostoevsky[words1_2]
    ][
        unigram2id_dostoevsky[word3]
#         условная вероятность генерируемого униграма после данного биграма
    ] = (trigrams_dostoevsky[trigram]/bigrams_dostoevsky[words1_2])

In [151]:
def generate(matrix, id2word, word2id, 
             n=100, start=['<start>','<start>']):
    text = []
    bigram = start
    current_idx = word2id[' '.join(bigram)]
    for i in range(n):
        
        chosen = np.random.choice(
#       выбирает среди столбиков (не строчек!)
        matrix.shape[1],
#       учитывает вероятности перехода стартового биграма с текущим
#       индексом в каждый униграм (столбик)
        p=matrix[current_idx]
        )
        
        text.append(
#           добавляет в текст униграм
            id2word[chosen])
#       если был выбран тег конца,  выбираем тег начала
        if id2word[chosen] == '<end>':
            bigram = start
            current_idx = word2id[' '.join(bigram)]
#       если любой другой униграм, то смотрим 
#       на последнее слово биграма, и из них стряпаем биграм
#       для следующего предсказания
        else:
          bigram = [bigram[1], id2word[chosen]]
          current_idx = word2id[' '.join(bigram)]
    return ' '.join(text)

In [157]:
print(generate(matrix=matrix_dostoevsky,
              id2word=id2unigram_dostoevsky,
              word2id=bigram2id_dostoevsky).replace('<end>', '\n'))
# о, французский

у нас исписываются совсем даже тихо промолвил степан трофимович 
 конечно вы на нас любуется ну ближе ближе 
 оставим это фр 
 сомнения нет что эти легендарные господа способны были ощущать и даже не притворено 
 вместо высших назначений она вдруг радостно усмехнулась 
 ну-с угодно вам выслушать 
 варвара петровна была скромно и по-всегдашнему одета во всё черное так бессменно одевалась она в изумлении 
 вспоминаю с улыбкой я не очень разговорчив изящен без изысканности удивительно скромен и в два часа опять нет 
 вы там каким-нибудь шефом меня представили 
 dieu qui est l&#224;-haut et qui est


In [153]:
import gensim
from gensim.models.phrases import Phrases, Phraser

Задание 2. (5 баллов) 
При помощи gensim.models.Phrases реализуйте byte-pair-encoding, про который говорилось на первом семинаре (https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Preprocessing.ipynb) 
А именно 1) возьмите любой текст; разбейте его на предложения, а каждое предложение разбейте на отдельные символы (не потеряйте пробелы) 2) обучите gensim.models.Phrases на полученных символьных предложениях 3) примените полученный нграммер к этим символьным предложениям 4) повторите 2 и 3 N количество раз, чтобы начали получаться целые слова или фразы
Параметры в gensim.models.Phrases влияют на количество получаемых нграммов после каждого прохода, поэтому не забудьте их настроить

Кажется, что, если добавить пробел в common_terms, слова склеиваются лучше. Но при этом повисают соседние пробелы.

In [169]:
txt = open('anna_karenina.txt', encoding='utf-8').read()
txt_sent = nltk.sent_tokenize(txt, 'english')
txt_sent = [' '.join(normalize(sent)) for sent in txt_sent]
txt_sent = [[char for char in sent] for sent in txt_sent]

In [172]:
def glue(stopchars=[]):
    phrasesmodel = gensim.models.Phrases(txt_sent, min_count=1,
                               threshold=-1,
                               delimiter=b'',
                            #    Write logs every progress_per sentence
                            #    progress_per=,
                            #    Score for given bi-gram, in the range -1 to 1
                               scoring='npmi',
                            #    Stopwords
                            #    common_terms=stopwords.words('russian')
                                common_terms=[' ']+stopchars
                                )
    print(list(phrasesmodel[txt_sent])[800])

    phrasesmodel2 = gensim.models.Phrases(phrasesmodel[txt_sent], min_count=1,
                                          delimiter=b'', threshold=-1, 
                                          scoring='npmi', common_terms=[' ']+stopchars)
    print(list(phrasesmodel2[phrasesmodel[txt_sent]])[800])

    phrasesmodel3 = gensim.models.Phrases(phrasesmodel2[phrasesmodel[txt_sent]], 
                                          min_count=1, delimiter=b'', threshold=-1, 
                                          scoring='npmi', common_terms=[' ']+stopchars)
    print(list(phrasesmodel3[phrasesmodel2[phrasesmodel[txt_sent]]])[800])

    phrasesmodel4 = gensim.models.Phrases(phrasesmodel3[phrasesmodel2[phrasesmodel[txt_sent]]], 
                                          min_count=1, threshold=-1, scoring='npmi', 
                                          delimiter=b'', common_terms=[' ']+stopchars)
    print(list(phrasesmodel4[phrasesmodel3[phrasesmodel2[phrasesmodel[txt_sent]]]])[800])

glue(stopchars=[])

['ну', ' ', 'та', 'к э', 'то', 'й м', 'ар', 'ки', ' ', 'к у', 'ст', 'ри', 'ца', 'м п', 'од', 'ай', ' ', 'а т', 'ам', ' ', 'ви', 'дн', 'о б', 'уд', 'ет']
['ну та', 'к это', 'й мар', 'ки к у', 'стри', 'цам п', 'одай', ' ', 'а там', ' ', 'видн', 'о буд', 'ет']
['ну так это', 'й марки к у', 'стрицам п', 'одай а там', ' ', 'видно буд', 'ет']
['ну так этой марки к у', 'стрицам подай а там', ' ', 'видно будет']


In [183]:
glue(stopchars=[x[0] for x in probas_dostoevsky.most_common(200) if len(x[0])==1])
# не очень хорошая оказалась идея, из-за неё развалилось ВИДНО
# но склеиваться стало быстрее

['ну т', 'а', 'к', ' ', 'эт', 'о', 'й м', 'а', 'рки к уст', 'риц', 'а', 'м п', 'о', 'дай', ' ', 'а', ' ', 'там', ' ', 'в', 'и', 'дн', 'о', ' ', 'буд', 'ет']
['ну так эт', 'о', 'й марки к уст', 'рицам п', 'о', 'дай а там', ' ', 'в', 'и', 'дно буд', 'ет']
['ну так этой марки к уст', 'рицам подай а там', ' ', 'в', 'и', 'дно будет']
['ну так этой марки к устрицам подай а там', ' ', 'в', 'и', 'дно будет']
